In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")
URM_test = sps.load_npz("URM_test_ens.npz")

URM_train_val = sps.load_npz("URM_train_val_ens.npz")

In [6]:
evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 527 ( 1.5%) Users that have less than 1 test interactions


In [7]:
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [8]:
def objective_function_KNN(optuna_trial):
                       
    similarity="tversky"
    
    recommender_instance = ItemKNNCFRecommender(URM_train_val)
    
    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 2, 30),
                   "shrink": optuna_trial.suggest_float("shrink", 5 , 30),
                   "feature_weighting": optuna_trial.suggest_categorical("feature_weighting", ["BM25", "TF-IDF"]),
                   "normalize": optuna_trial.suggest_categorical("normalize", [True, False])
                  }
    
    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0.1, 0.5, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0.4, 2.4, log=False)
        full_hyperp["normalize"] = True 

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])
        
    
    recommender_instance.fit(**full_hyperp)
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)   
    
    # Save MAP alongside RECALL
    trial_results = {
        "RECALL": eval_res["RECALL"][10],
        "MAP": eval_res["MAP"][10]  # Save MAP for reference
    }
    
    # Save the MAP value to the optuna trial user attributes for later use
    optuna_trial.set_user_attr("MAP", trial_results["MAP"])
    
    # Return RECALL for optimization
    return trial_results["MAP"]

In [9]:
class SaveResults(object):
    def __init__(self, file_path=None):
        self.file_path = file_path
        self.results = []
    
    def __call__(self, optuna_study, optuna_trial):
        trial_result = optuna_trial.params.copy()
        trial_result["iteration"] = optuna_trial.number
        trial_result["RECALL"] = optuna_trial.value
        
        # Retrieve the MAP metric from user attributes
        trial_result["MAP"] = optuna_trial.user_attrs.get("MAP")
        
        self.results.append(trial_result)
        
        # Convert to DataFrame and save to CSV
        results_df = pd.DataFrame(self.results)
        results_df.to_csv(self.file_path, index=False)


In [10]:
optuna_study = op.create_study(direction="maximize")
        
save_results_tanimoto = SaveResults("result_experiments/ItemKNNCFRecommender/results_tversky_optuna_map.csv")
        
optuna_study.optimize(objective_function_KNN,
                      callbacks=[save_results_tanimoto],
                      n_trials = 500)

[I 2025-01-06 23:56:04,132] A new study created in memory with name: no-name-1b252f92-2774-4189-9b7f-ba0ad057e4ac


Similarity column 38121 (100.0%), 4520.34 column/sec. Elapsed time 8.43 sec
EvaluatorHoldout: Processed 35209 (100.0%) in 9.13 sec. Users per second: 3858


[I 2025-01-06 23:56:21,869] Trial 0 finished with value: 0.04211992491998874 and parameters: {'topK': 7, 'shrink': 20.313201417217225, 'feature_weighting': 'TF-IDF', 'normalize': False, 'tversky_alpha': 0.3691069118125733, 'tversky_beta': 2.145377446930075}. Best is trial 0 with value: 0.04211992491998874.


Similarity column 38121 (100.0%), 4466.93 column/sec. Elapsed time 8.53 sec
EvaluatorHoldout: Processed 35209 (100.0%) in 9.07 sec. Users per second: 3883


[I 2025-01-06 23:56:39,635] Trial 1 finished with value: 0.040162573422107835 and parameters: {'topK': 4, 'shrink': 28.101187900121026, 'feature_weighting': 'TF-IDF', 'normalize': False, 'tversky_alpha': 0.15195064865896224, 'tversky_beta': 0.48109474639477956}. Best is trial 0 with value: 0.04211992491998874.


...

[I 2025-01-07 02:47:40,547] Trial 497 finished with value: 0.042556846486309 and parameters: {'topK': 3, 'shrink': 20.871824283719537, 'feature_weighting': 'TF-IDF', 'normalize': True, 'tversky_alpha': 0.19951089927173585, 'tversky_beta': 2.325159111930219}. Best is trial 434 with value: 0.042848370974771816.


Similarity column 38121 (100.0%), 4678.85 column/sec. Elapsed time 8.15 sec
EvaluatorHoldout: Processed 35209 (100.0%) in 8.55 sec. Users per second: 4118


[I 2025-01-07 02:47:57,422] Trial 498 finished with value: 0.04270242840146683 and parameters: {'topK': 5, 'shrink': 22.201082689722007, 'feature_weighting': 'TF-IDF', 'normalize': True, 'tversky_alpha': 0.20503482677460624, 'tversky_beta': 2.355544613706662}. Best is trial 434 with value: 0.042848370974771816.


Similarity column 38121 (100.0%), 4693.65 column/sec. Elapsed time 8.12 sec
EvaluatorHoldout: Processed 35209 (100.0%) in 8.49 sec. Users per second: 4146


[I 2025-01-07 02:48:14,211] Trial 499 finished with value: 0.04274859151722869 and parameters: {'topK': 4, 'shrink': 21.519899419966727, 'feature_weighting': 'TF-IDF', 'normalize': True, 'tversky_alpha': 0.183988307788595, 'tversky_beta': 2.3958022535967247}. Best is trial 434 with value: 0.042848370974771816.


In [11]:
optuna_study.best_params, optuna_study.best_value

({'topK': 4,
  'shrink': 21.349848360360596,
  'feature_weighting': 'TF-IDF',
  'normalize': True,
  'tversky_alpha': 0.21211620571743547,
  'tversky_beta': 2.2770208408345005},
 0.042848370974771816)

In [15]:
pd.DataFrame(save_results_tanimoto.results).sort_values(by=["MAP"], ascending=False)

,topK,shrink,feature_weighting,normalize,tversky_alpha,tversky_beta,iteration,RECALL,MAP
96,6,21.589349,TF-IDF,False,0.167252,1.722013,96,0.118551,0.028130
220,6,22.217756,none,False,0.227589,1.788536,220,0.118239,0.028126
102,6,21.079349,TF-IDF,False,0.196247,1.733155,102,0.118240,0.028112
83,6,22.622420,TF-IDF,False,0.255611,1.770004,83,0.118308,0.028104
249,6,20.613082,none,False,0.235928,1.767350,249,0.118309,0.028103
...,...,...,...,...,...,...,...,...,...
6,19,18.287544,BM25,True,0.702835,0.703088,6,0.103331,0.023864
5,47,8.640345,BM25,False,0.346342,1.396634,5,0.102509,0.023736
3,42,8.196521,BM25,False,0.173447,0.219287,3,0.099767,0.022978
43,2,22.286228,TF-IDF,False,0.320202,0.102723,43,0.100111,0.022212


In [18]:
save_results_tanimoto.results[96]

{'topK': 6,
 'shrink': 21.589349496863413,
 'feature_weighting': 'TF-IDF',
 'normalize': False,
 'tversky_alpha': 0.16725184866550824,
 'tversky_beta': 1.722012798691049,
 'iteration': 96,
 'RECALL': 0.11855115150555026,
 'MAP': 0.02812995231879061}

In [18]:
save_results_cosine.results_df['result'].max() 

0.0263499559387598

In [33]:
save_results_adjusted.results_df.to_csv("result_experiments/ItemKNNCFRecommender/results_adjusted.csv")